In [40]:
import hkvfewspy as fews
import pandas as pd
import json
import numpy as np

In [2]:
fews.__version__

'0.6.3'

In [3]:
# pi url
pi_url = 'https://db.dmhoutribdijk.nl/FewsWebServices/fewspiservice?wsdl'

# location / parameter combinations to evaluate
filter_id = 'f_ruw_ongevalideerd.STA'
parameter_ids = ['status.etro', 'status.isa','status.ysi', 'status.aqd', 'status.vec', 'status.rdi']
location_ids = ['FL65','FL66','FL67','FL68','FL69','FL70','FL65_A','FL65_B','FL65_C','FL69_A','FL69_B','FL69_C']

# status excel file
status_overview = r"D:/Projects/Pr/3916.10/Werkmap/status_overzicht.csv"

# output file
file = r'D:/Projects/Pr/3916.10/Werkmap/prtg_out.json'

In [4]:
json.dumps(location_ids)

'["FL65", "FL66", "FL67", "FL68", "FL69", "FL70", "FL65_A", "FL65_B", "FL65_C", "FL69_A", "FL69_B", "FL69_C"]'

In [5]:
pi = fews.Pi()
pi.setClient(pi_url)

In [6]:
filters = pi.getFilters()

In [7]:
filters

,f_ruw_ongevalideerd_ADC,f_ruw_ongevalideerd_ADV,f_ruw_ongevalideerd_ALT,f_ruw_ongevalideerd_AQD,f_ruw_ongevalideerd_MPP,f_ruw_ongevalideerd_STA,f_ruw_ongevalideerd_STB,f_ruw_gevalideerd_ADC,f_ruw_gevalideerd_ADV,f_ruw_gevalideerd_ALT,f_ruw_gevalideerd_AQD,f_ruw_gevalideerd_MPP,f_ruw_gevalideerd_STB,f_meteo,f_dms
description,,,,,,,,,,,,,,,
id,f_ruw_ongevalideerd.ADC,f_ruw_ongevalideerd.ADV,f_ruw_ongevalideerd.ALT,f_ruw_ongevalideerd.AQD,f_ruw_ongevalideerd.MPP,f_ruw_ongevalideerd.STA,f_ruw_ongevalideerd.STB,f_ruw_gevalideerd.ADC,f_ruw_gevalideerd.ADV,f_ruw_gevalideerd.ALT,f_ruw_gevalideerd.AQD,f_ruw_gevalideerd.MPP,f_ruw_gevalideerd.STB,f_meteo,f_dms
name,ADC,ADV,ALT,AQD,MPP,STA,STB,ADC,ADV,ALT,AQD,MPP,STB,KNMI-meteo,DMS-data


In [101]:
# create rounded reference T0 
T0 = pd.Timestamp.now().round('60min').tz_localize('Europe/Amsterdam')
T0 = T0.tz_convert('Etc/GMT') # to timezone as in FEWS

# and start- / end-date.
start_date = T0 - pd.Timedelta(hours=3)
end_date = T0 - pd.Timedelta(hours=2, minutes=10)

In [102]:
T0

Timestamp('2019-05-29 12:00:00+0000', tz='Etc/GMT')

In [103]:
start_date, end_date

(Timestamp('2019-05-29 09:00:00+0000', tz='Etc/GMT'),
 Timestamp('2019-05-29 09:50:00+0000', tz='Etc/GMT'))

In [92]:
# set query parameters
query = pi.setQueryParameters()

query.clientTimeZone('Etc/GMT')
query.filterId(filter_id)
query.parameterIds(parameter_ids)
query.locationIds(location_ids)
query.startTime(start_date)
query.endTime(end_date)

# request timeseries
df = pi.getTimeSeries(queryParameters=query, )

In [93]:
# mask nodata values using the flag value
df.value.mask(df.flag == 8, inplace=True)

In [94]:
df.head()

,moduleInstanceId,qualifierId,parameterId,units,locationId,stationName,flag,value
date,,,,,,,,
2019-05-29 09:00:00+00:00,ImportStatus,,status.aqd,-,FL65,FL65 Marker Wadden Zuiderstrand,0,0
2019-05-29 09:00:00+00:00,ImportStatus,,status.aqd,-,FL65_A,FL65_A Marker Wadden Zuiderstrand,0,0
2019-05-29 09:00:00+00:00,ImportStatus,,status.aqd,-,FL65_B,FL65_B Marker Wadden Zuiderstrand,0,-1
2019-05-29 09:00:00+00:00,ImportStatus,,status.aqd,-,FL65_C,FL65_C Marker Wadden Zuiderstrand,0,0
2019-05-29 09:00:00+00:00,ImportStatus,,status.aqd,-,FL66,FL66 Marker Wadden Noorderstrand,0,0


In [95]:
df.tail()

,moduleInstanceId,qualifierId,parameterId,units,locationId,stationName,flag,value
date,,,,,,,,
2019-05-29 12:00:00+00:00,ImportStatus,,status.ysi,-,FL69,FL69 IJsselmeer 2,8,NaN
2019-05-29 12:00:00+00:00,ImportStatus,,status.ysi,-,FL69_A,FL69_A IJsselmeer 2,8,NaN
2019-05-29 12:00:00+00:00,ImportStatus,,status.ysi,-,FL69_B,FL69_B IJsselmeer 2,8,NaN
2019-05-29 12:00:00+00:00,ImportStatus,,status.ysi,-,FL69_C,FL69_C IJsselmeer 2,8,NaN
2019-05-29 12:00:00+00:00,ImportStatus,,status.ysi,-,FL70,FL70 IJsselmeer 1,8,NaN


In [73]:
# apply join on parameter names
lookup_pars = pi.getParameters(filterId=filter_id)
lookup_pars = lookup_pars.T[['id','name']]
df_merge = df.reset_index().merge(lookup_pars, left_on='parameterId', right_on='id')

In [74]:
# get overview at which location parameter combination status information is expected
df_overzicht = pd.read_csv(status_overview, index_col=0)

In [75]:
# parse to combination data
combinations = []
for row in df_overzicht.iterrows():
    for idx, item in enumerate(row[1]):
        pair = {
            'locId':row[0],
            'parId':row[1].index[idx],
            'monitor':item
        }
        combinations.append(pair)
        
df_monitor = pd.DataFrame.from_dict(combinations)        

In [77]:
# apply inner-join on retrieved status information and overview of status setting
df_status = df_merge.merge(df_monitor, left_on=['locationId', 'name'], right_on=['locId', 'parId'])
df_status = df_status[['date', 'locationId', 'name', 'monitor','value']]

In [78]:
# only select location parameter combinations where there is monitoring activated
df_msg = df_status[df_status['monitor'] == 1]

# filter where status is NaN or 0 (only value -1 is valid)
msg_filter = np.array([any(tup) for tup in zip(df_msg.value.isnull(), df_msg.value == 0)])

In [84]:
# add columns for prtg
df_msg.loc[:,'channel'] = df_msg['locationId']+ ' - ' + df_msg['name']
df_msg.loc[:, 'value_error'] = 0
df_msg.loc[msg_filter, 'value_error'] = 2

df_msg.loc[:,'limitmaxerror'] = 1
df_msg.loc[:,'limitmode'] = 1
df_msg.loc[:,'limiterrormsg'] = 'Status-tijdreeks locatie-sensor combinatie wordt correct ontvangen (waarde -1)'
df_msg.loc[msg_filter,'limiterrormsg'] = 'Status-tijdreeks wordt niet ontvangen (waarde 0 of NaN) terwijl de locatie-sensor combinatie wel is aangemeld. ({})'.format(df_msg.date.dt.strftime('%Y-%m-%d %H:%M'))

In [85]:
df_msg.head()

,date,locationId,name,monitor,value,channel,value_error,limitmaxerror,limitmode,limiterrormsg
12,2019-05-29 07:00:00+00:00,FL65_B,Status AQD,True,-1,FL65_B - Status AQD,0,1,1,Status-tijdreeks locatie-sensor combinatie wor...
13,2019-05-29 07:10:00+00:00,FL65_B,Status AQD,True,-1,FL65_B - Status AQD,0,1,1,Status-tijdreeks locatie-sensor combinatie wor...
14,2019-05-29 07:20:00+00:00,FL65_B,Status AQD,True,-1,FL65_B - Status AQD,0,1,1,Status-tijdreeks locatie-sensor combinatie wor...
15,2019-05-29 07:30:00+00:00,FL65_B,Status AQD,True,-1,FL65_B - Status AQD,0,1,1,Status-tijdreeks locatie-sensor combinatie wor...
16,2019-05-29 07:40:00+00:00,FL65_B,Status AQD,True,-1,FL65_B - Status AQD,0,1,1,Status-tijdreeks locatie-sensor combinatie wor...


In [81]:
# filter columns to output
df_msg_out = df_msg[['channel','value_error','limitmaxerror', 'limitmode','limiterrormsg']]
df_msg_out.columns = ['channel','value','limitmaxerror', 'limitmode','limiterrormsg']

# save to file
error_out = {"prtg":{"result": df_msg_out.to_dict(orient='records')}}
with open(file, 'w') as f:
    print(json.dumps(error_out), file=f)  # Python 3.x

In [ ]:
# import altair as alt
# # setup an interactive line chart for the year 1953-1955
# line_chart = alt.Chart(data=df_msg, width=100, height=100).mark_line(
#     point=True
# ).encode(
#     x='date',
#     y='value',
#     color=alt.Color('locationId', scale=alt.Scale(scheme='category20')),
#     tooltip=['value', 'locationId', alt.Tooltip('date', format='%Y-%m-%d %H:%M')]
# ).facet(
#     column='name',
#     row='locationId'
# )

# # diplay the chart
# line_chart